<a href="https://colab.research.google.com/github/mertkaya1033/MNIST_digit_classify/blob/master/MNIST_digit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST Handwritten Digit Classification


## Imports and Helpers

In [31]:
import time
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable

from typing import Tuple

# https://stackoverflow.com/questions/39832721/meaning-of-self-dict-self-in-a-class-definition
class Config(dict):
  """A class that allows attribute access and item access."""
  def __init__(self):
    self.__dict__ = self
  

## MNIST Digit Classification Trainer

In [32]:
class MNIST_Classify():
  """Downloads the MNIST dataset, and trains models that predict the handwritten
  digit written on the provided 28x28 image."""

  _training_data = None
  _test_data = None

  @staticmethod
  def load_data(root: str = "data") -> Tuple[Dataset, Dataset]:
    """Downloads MNIST dataset into local file system, into the root directory. 
    Loads the dataset into memory.
    
    Parameters:
      root (str) -- the directory to download the dataset. (default: "data")
    
    Returns:
      tuple --  (training dataset, test dataset)
    """
    if MNIST_Classify._training_data is None:
      MNIST_Classify._training_data = datasets.MNIST(
          root=root,
          train=True,
          download=True,
          transform=ToTensor()
      )
    if MNIST_Classify._test_data is None:
      MNIST_Classify._test_data = datasets.MNIST(
          root=root,
          train=False,
          download=True,
          transform=ToTensor()
      )
    return MNIST_Classify._training_data, MNIST_Classify._test_data
    
  
  def _prepare_data(self, args: Config) -> None:
    """Prepares the dataset for training.

    Parameters:
        args (Config) --  hyperparameters of the model being trained
    """
    print("Preparing Data...")
    MNIST_Classify.load_data()

    self._train_dataloader = DataLoader(MNIST_Classify._training_data, batch_size=args.batch_size, shuffle=True)
    self._test_dataloader = DataLoader(MNIST_Classify._test_data, batch_size=args.batch_size, shuffle=True)

  def train_model(self, args: Config):
    """Trains an instance of a model provided in args with the given hyperparameters
    for the task of classifying handwritten digits using the MNIST dataset.

    Parameters:
        args (Config) --  stores the model and the hyperparameters in which the 
                          model instance should be trained

    Returns:
        The trained model instance
    """
    print("===================", args.experiment_name, "===================")

    
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    
    self._prepare_data(args)
    
    num_digits = 10
    num_channels = 1
    image_size = MNIST_Classify._training_data.data.size()[1] #28x28
    num_batches = len(self._train_dataloader)

    mnist_model = args.Model(image_size, num_digits, num_channels, args)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=mnist_model.parameters(), lr=args.learn_rate)
    
    train_losses = []
    val_losses = []
    val_accuracies = []
    print("Beginning Training...")
    start_time = time.time()
    for epoch in range(args.epochs):
      
      mnist_model.train()
      losses = []
      
      # Training
      for i, (input, label) in enumerate(self._train_dataloader):
        optimizer.zero_grad()
        
        predictions = mnist_model(input)
        loss = loss_func(predictions, label)

        loss.backward()
        optimizer.step()

        losses.append(loss.data.item())
      
      mnist_model.eval()
      avg_loss = np.mean(losses)
      train_losses.append(avg_loss)
      time_elapsed = time.time() - start_time
      print(f"Epoch [{epoch + 1}/{args.epochs}], Loss: {avg_loss:.4f}, Time(s): {time_elapsed:.2f}")

      # Performance with validation set
      correct = 0.0
      total = 0.0
      losses = []
      for i, (input, label) in enumerate(self._test_dataloader):
        predictions = mnist_model(input)
        loss = loss_func(predictions, label)
        losses.append(loss.data.item())
        _, predicted = torch.max(input=predictions.data, dim=1)
        
        correct += (predicted == label).sum()
        total += label.size(0)

      
      val_loss = np.mean(losses)      
      val_acc = 100.0 * correct / total
      val_losses.append(val_loss)
      val_accuracies.append(val_acc)
      time_elapsed = time.time() - start_time
      print(f"Epoch [{epoch + 1}/{args.epochs}], Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}, Time(s): {time_elapsed:.2f}")


    return mnist_model, [train_losses, val_losses, val_accuracies]


## 1. Convolutional Neural Network 

### The Model

This model consists of 2 convolution blocks, and a linear classifier. The convolution blocks are the following layers in sequence:
* Convolution layer
* Max pooling layer
* Batch Norm layer
* ReLU activation layer

In [33]:
class MNIST_CNN(nn.Module):
  """
  A convolutional neural network
  """
  def __init__(self, image_size, num_classes, num_in_channels, train_args) -> None:
      super().__init__()

      kernel = train_args.kernel
      num_filters = train_args.num_filters

      padding = kernel // 2

      self.convBlock1 = nn.Sequential(
          nn.Conv2d(
              in_channels=num_in_channels, 
              out_channels=num_filters,
              kernel_size=kernel,
              padding=padding),
          nn.MaxPool2d(kernel_size=2),
          nn.BatchNorm2d(num_features=num_filters),
          nn.ReLU())
      
      self.convBlock2 = nn.Sequential(
          nn.Conv2d(
              in_channels=num_filters, 
              out_channels=num_filters*2,
              kernel_size=kernel,
              padding=padding),
          nn.MaxPool2d(kernel_size=2),
          nn.BatchNorm2d(num_features=num_filters*2),
          nn.ReLU())
      
      self.flatten = nn.Flatten()

      in_features = (num_filters*2)*(image_size//4)*(image_size//4)

      self.linearBlock = nn.Sequential(
          nn.Linear(in_features=in_features, out_features=num_classes),
      )
      

  def forward(self, x):
    convolved1 = self.convBlock1(x)
    convolved2 = self.convBlock2(convolved1)
    flattened = self.flatten(convolved2)
    prediction = self.linearBlock(flattened)
    return prediction

### Training

In [34]:
# CONFIG
cnn_args = Config()
cnn_args_dict = {
    "experiment_name": "MNIST_digit_classification_CNN",
    "Model": MNIST_CNN,
    "seed": 0,
    "epochs": 25,
    "learn_rate": 0.001,
    "batch_size": 64,
    "kernel": 5,
    "num_filters": 16,
}
cnn_args.update(cnn_args_dict)

trainer = MNIST_Classify()
cnn_model, _ = trainer.train_model(cnn_args)

=================== MNIST_digit_classification_CNN ===================
Preparing Data...
Beginning Training...
Epoch [1/25], Loss: 0.1036, Time(s): 34.64
Epoch [1/25], Val Loss: 0.0396, Val Acc: 98.78, Time(s): 37.46
Epoch [2/25], Loss: 0.0358, Time(s): 71.41
Epoch [2/25], Val Loss: 0.0313, Val Acc: 98.95, Time(s): 74.24
Epoch [3/25], Loss: 0.0253, Time(s): 108.34
Epoch [3/25], Val Loss: 0.0416, Val Acc: 98.71, Time(s): 111.21
Epoch [4/25], Loss: 0.0184, Time(s): 145.26
Epoch [4/25], Val Loss: 0.0271, Val Acc: 99.09, Time(s): 148.10
Epoch [5/25], Loss: 0.0139, Time(s): 182.20
Epoch [5/25], Val Loss: 0.0302, Val Acc: 98.93, Time(s): 185.01
Epoch [6/25], Loss: 0.0137, Time(s): 218.31
Epoch [6/25], Val Loss: 0.0280, Val Acc: 99.23, Time(s): 221.12
Epoch [7/25], Loss: 0.0097, Time(s): 254.38
Epoch [7/25], Val Loss: 0.0271, Val Acc: 99.20, Time(s): 257.15
Epoch [8/25], Loss: 0.0068, Time(s): 290.42
Epoch [8/25], Val Loss: 0.0307, Val Acc: 99.13, Time(s): 293.21
Epoch [9/25], Loss: 0.0085, T

## 2. AutoEncoders